<h1> Package Import</h1>

In [2]:
import requests; import json; import random; import openpyxl; import xlwt
import numpy as np; import pandas as pd; import matplotlib.pylab as plt

<h1> Functions </h1>

In [3]:
def DB_CONNECT(i):
    url = 'https://www.dhlottery.co.kr/common.do?method=getLottoNumber&drwNo=' + str(i)
    res = requests.get(url)
    connection = res.json()
    return connection

def DF_LOTTERY(number_lotteryCount):
    lottos_dics = {}
    totSellamnt, firstAccumamnt, firstWinamnt, firstPrzwnerCo = [], [], [], []
    drwtNo1, drwtNo2, drwtNo3, drwtNo4, drwtNo5, drwtNo6, bnusNo = [], [], [], [], [], [], []
    
    for i in range(1, number_lotteryCount+1):
        connection = DB_CONNECT(i)
        totSellamnt.append(connection['totSellamnt'])
        firstAccumamnt.append(connection['firstAccumamnt'])
        firstWinamnt.append(connection['firstWinamnt'])
        firstPrzwnerCo.append(connection['firstPrzwnerCo'])
        drwtNo1.append(connection['drwtNo1'])
        drwtNo2.append(connection['drwtNo2'])
        drwtNo3.append(connection['drwtNo3'])
        drwtNo4.append(connection['drwtNo4'])
        drwtNo5.append(connection['drwtNo5'])
        drwtNo6.append(connection['drwtNo6'])
        bnusNo.append(connection['bnusNo'])
        lottos_dics = {'totSellamnt': totSellamnt, 'firstAccumamnt': firstAccumamnt, 'firstWinamnt': firstWinamnt, 'firstPrzwnerCo':firstPrzwnerCo, 'drwtNo1': drwtNo1, 'drwtNo2': drwtNo2, 'drwtNo3': drwtNo3, 'drwtNo4': drwtNo4, 'drwtNo5': drwtNo5, 'drwtNo6': drwtNo6, 'bnusNo': bnusNo}
    
    DF_LOTTERY = pd.DataFrame(lottos_dics)
    return DF_LOTTERY

def DF_RANDOM(number_avgsaleVolume):
    def GENERATE_RANDOM_LOTTERY(number_avgsaleVolume):
        for _ in range(number_avgsaleVolume):
            number_random = sorted(random.sample(range(1, 46), 6))
            number_bnus = random.choice(list(set(range(1, 46)) - set(number_random)))
            yield number_random + [number_bnus]
    DF_RANDOM = pd.DataFrame(GENERATE_RANDOM_LOTTERY(number_avgsaleVolume), columns = ['drwtNo1','drwtNo2','drwtNo3','drwtNo4','drwtNo5','drwtNo6','bnusNo'])
    return DF_RANDOM

def AVERAGE_GAMECOUNT(df_input):
    average_saleAmnt = df_input['totSellamnt'].mean()
    average_gameCount = int(average_saleAmnt/1000)
    print('Average Sales Volume : {:,.0f}'.format(average_gameCount))
    return average_gameCount
    
def STATISTICS_NUMBERINFO(df_input):
    count_row, count_col = df_input.shape[0], df_input.shape[1]
    df_numberInfo = pd.DataFrame(np.zeros((1, 45)), columns = range(1, 46), index = ['Frequency'])
    for i in range(0, count_row):
        for j in range(1, 46):
            if df_input.iloc[i]['drwtNo1'] == j: df_numberInfo[j] += 1
            elif df_input.iloc[i]['drwtNo2'] == j: df_numberInfo[j] += 1
            elif df_input.iloc[i]['drwtNo3'] == j: df_numberInfo[j] += 1
            elif df_input.iloc[i]['drwtNo4'] == j: df_numberInfo[j] += 1
            elif df_input.iloc[i]['drwtNo5'] == j: df_numberInfo[j] += 1
            elif df_input.iloc[i]['drwtNo6'] == j: df_numberInfo[j] += 1
            elif df_input.iloc[i]['bnusNo'] == j: df_numberInfo[j] += 1
    return df_numberInfo

def STATISTIC_DATAINFO(df_input):
    def DIC_CONVERT(df_input, name_var):
        mean, std, min, q1, median, q3, max = df_input[name_var].mean(), df_input[name_var].std(), df_input[name_var].min(), df_input[name_var].quantile(0.25), df_input[name_var].median(), df_input[name_var].quantile(0.75), df_input[name_var].max()
        dic_function = {'mean' : '{:,.3f}'.format(mean), 'standard deviation' : '{:,.3f}'.format(std), 'min' : '{:,.0f}'.format(min), 'Q1' : '{:,.0f}'.format(q1), 'median' : '{:,.0f}'.format(median), 'Q3' : '{:,.0f}'.format(q3), 'max' : '{:,.0f}'.format(max)}
        return dic_function
    df_return = pd.DataFrame()
    col_datainfo = ['totSellamnt', 'firstAccumamnt', 'firstWinamnt', 'firstPrzwnerCo']
    for col in col_datainfo:
        df_store = pd.DataFrame(DIC_CONVERT(df_input, col), index = [col])
        df_return = pd.concat([df_return, df_store], ignore_index = False)
    df_datainfo = df_return.transpose()
    return df_datainfo

def RECOMMEND_NUMBER_FROM_RANDOM(df_lottery, df_random):
    def REMOVE_DUPLICATE_WITH_LOTTERY(df_lottery, df_random):
        df_lottery_dupCheck = df_lottery.copy().drop(columns = ['totSellamnt', 'firstAccumamnt', 'firstWinamnt', 'firstPrzwnerCo'])
        tuple_dupCheck = set(map(tuple, df_lottery_dupCheck.values))
        df_random['is_duplicate'] = df_random.apply(lambda row : tuple(row.values) in tuple_dupCheck, axis = 1)

        df_random = df_random[~df_random['is_duplicate']]
        df_random.drop(columns = ['is_duplicate'], inplace = True)
        return df_random
    def COUNT_DUPLICATE_AMONG_RANDOM(df_random):
        df_random_analysis = df_random.copy()
        df_random_analysis['count'] = 0
        dict_count = {}
        for i, row in df_random_analysis.iterrows():
            row_tuple = tuple(row)
            if row_tuple in dict_count: dict_count[row_tuple] += 1
            else: dict_count[row_tuple] = 1
            df_random_analysis.at[i, 'count'] = dict_count[row_tuple] - 1
        df_random_analysis_dupCount = df_random_analysis.sort_values(by = 'count', ascending = True)
        
        return df_random_analysis_dupCount

    random_remove_dup_with_Lottery = REMOVE_DUPLICATE_WITH_LOTTERY(df_lottery, df_random)
    random_count_dup_among_random = COUNT_DUPLICATE_AMONG_RANDOM(random_remove_dup_with_Lottery)
    random_dupCount_0 = random_count_dup_among_random[random_count_dup_among_random['count'] == 0.0]
    random_dupCount_no0 = random_count_dup_among_random[random_count_dup_among_random['count'] != 0.0]
    
    return random_dupCount_0, random_dupCount_no0

<h2> Initialize Lottery Data Set </h2>
number = 로또 회차 입력 (전 주 회차)
average_saleVolume = 전 주까지 평균 판매액 / 1000 => 전 주까지 평균 판매 게임

In [4]:
number = 1113

In [5]:
df_lottery = DF_LOTTERY(number)

df_numberInfo = STATISTICS_NUMBERINFO(df_lottery)
df_dataInfo = STATISTIC_DATAINFO(df_lottery)

average_gameCount = AVERAGE_GAMECOUNT(df_lottery)
df_random = DF_RANDOM(average_gameCount)

df_random_dupCount_0, df_random_dupCount_no0 = RECOMMEND_NUMBER_FROM_RANDOM(df_lottery, df_random)

Average Sales Volume : 67,110,768


/var/folders/5p/v2x2x94x4z1bzq3vb084rzbc0000gn/T/ipykernel_2467/122730688.py:79: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_random.drop(columns = ['is_duplicate'], inplace = True)


<h2> Statistics of Lottery </h2>

In [6]:
df_numberInfo

,1,2,3,4,5,6,7,8,9,10,...,36,37,38,39,40,41,42,43,44,45
Frequency,185.0,174.0,177.0,181.0,162.0,177.0,177.0,162.0,140.0,175.0,...,173.0,177.0,177.0,177.0,180.0,154.0,167.0,192.0,174.0,178.0


In [7]:
df_dataInfo

,totSellamnt,firstAccumamnt,firstWinamnt,firstPrzwnerCo
mean,"67,110,768,286.613","12,809,133,802.927","2,687,568,924.446",7.817
standard deviation,"22,283,753,149.394","8,773,485,875.016","2,441,747,295.935",4.418
min,"3,681,782,000",0,0,0
Q1,"49,582,758,000",0,"1,558,378,334",5
median,"60,983,308,000","13,759,405,875","2,077,279,594",7
Q3,"80,933,117,000","18,976,489,128","3,051,105,610",10
max,"260,856,392,000","43,624,374,375","40,722,959,400",50


<h2> Random Generation Results </h2>

In [12]:
print(f'(1) Shape of duplicated 0 Result : {df_random_dupCount_0.shape}\n(2) Shape of duplicated without 0 Result : {df_random_dupCount_no0.shape}');print('-'*100)
print(f'[Duplicated Count among Random Generated Numbers] Duplicated without 0\n{df_random_dupCount_no0["count"].value_counts()}')

(1) Shape of duplicated 0 Result : (60493684, 8)
(2) Shape of duplicated without 0 Result : (6616848, 8)
----------------------------------------------------------------------------------------------------
[Duplicated Count among Random Generated Numbers] Duplicated without 0
count
1    6168476
2     425104
3      22298
4        936
5         33
6          1
Name: count, dtype: int64


<h2> Recommand Number </h2>

In [17]:
threshold = 5 # Choose the Threshold you want to rely on
threshold_based_ = df_random_dupCount_no0[df_random_dupCount_no0['count'] == threshold]

In [19]:
number_recommend = threshold_based_.sample(n = 5)
print(number_recommend)

          drwtNo1  drwtNo2  drwtNo3  drwtNo4  drwtNo5  drwtNo6  bnusNo  count
47843692       10       25       26       27       30       43      28      5
61491699        2       29       32       33       34       37      22      5
53702856        3        6       22       23       26       42      45      5
64373516       14       20       26       28       29       45      34      5
64530413       17       21       32       40       42       43      38      5
